In [29]:
import pandas as pd
import numpy as np
import cPickle
import plotly.graph_objs as go
import plotly.plotly as py
fom plotly.offline import init_notebook_mode, iplot
sign_in('johnnysand7', 'jfb78m4uw3')
init_notebook_mode()

In [42]:
import matplotlib.pyplot as plt
%matplotlib inline

# `item_data` 

In [3]:
item_data = pd.read_csv("/home/ubuntu/game_data.csv")

In [4]:
item_data["price_int"] = item_data["price"].round().astype(int)

In [5]:
with open("game_reviews.json") as f:
    game_reviews = f.readlines()
game_reviews = map(lambda x: x.strip(), game_reviews)
game_reviews = "[" + ','.join(game_reviews) + "]"
reviews_df = pd.read_json(game_reviews)
del reviews_df["_id"]
reviews_df.rename(columns={"review":"community_review"}, inplace=True)
reviews_add = pd.merge(item_data, reviews_df, on="appid")

In [6]:
reviews_df.head()

,appid,community_review
0,200710,95
1,71340,93
2,10,97
3,219150,97
4,20,84


In [40]:
with open("meta_reviews.json") as f:
    meta_reviews = f.readlines()
meta_reviews = map(lambda x: x.strip(), meta_reviews)
meta_reviews = "[" + ','.join(meta_reviews) + "]"
meta_df = pd.read_json(meta_reviews)
del meta_df["_id"]
meta_df.rename(columns={"review":"meta_review"}, inplace=True)
reviews_add = reviews_add.merge(meta_df, on="appid")
del reviews_add["Unnamed: 0"]

In [8]:
reviews_add.to_csv("item_data_community_meta_reviews")

Example of some of the scraped review data looks like

In [41]:
reviews_add.head()

,appid,genre,price,release_year,reviews,game_age,review_nums,price_int,item_id,community_review,meta_review_x,meta_review_y,meta_review
0,7830,Strategy,4.99,2009,Very Positive,7,7,5,7830,87,80,80,80
1,97330,Strategy,9.99,2012,Very Positive,4,7,10,97330,89,77,77,77
2,108710,Action,29.99,2012,Very Positive,4,7,30,108710,91,83,83,83
3,209000,Action,19.99,2013,Very Positive,3,7,20,209000,85,74,74,74
4,72850,RPG,19.99,2011,Very Positive,5,7,20,72850,92,94,94,94


# user-item pairs

In [10]:
user_items = pd.read_csv("/home/ubuntu/final_sf.csv")

Bin playtime to get an explicit rating

In [11]:
user_items["rating"] = pd.cut(user_items["playtime_forever"],\
                                          bins=[-1, 60, 120, 180, 240, 300, 10e10], \
                                          labels=[0,1,2,3,4,5])

In [12]:
user_items["rating"] = user_items["rating"].astype(int)

In [13]:
def check_reviews(df, apps):
    game_dict = {"appid":[], "review":[]}
    for app in apps:
        summed_reviews = df[df["appid"]==app]["rating"].sum()
        number_played = float((df[df["appid"]==app]["rating"]!=0).sum())
        out_of_10 = (summed_reviews / number_played) / 5
        game_dict["appid"].append(app)
        game_dict["review"].append(out_of_10)
    output_df = pd.DataFrame(game_dict)
    return  output_df

In [14]:
my_reviews = check_reviews(user_items, item_data.appid.tolist())

In [17]:
my_reviews = my_reviews.sort_values(by="appid")
my_reviews.reset_index(inplace=True, drop=True)
my_reviews.review = my_reviews.review * 100

In [20]:
review_data = pd.read_csv("item_data_community_meta_reviews")
del review_data["Unnamed: 0"]
del review_data["Unnamed: 0.1"]

In [21]:
review_data = review_data.sort_values(by="appid")
review_data.reset_index(drop=True, inplace=True)
review_data = review_data.merge(my_reviews, on="appid")

See how the playtime rating fairs when compared to the community reviews and metacritic reviews

In [34]:
review_data.sort_values(by="review", inplace=True)
review_data.reset_index(drop=True, inplace=True)

In [35]:
trace0 = go.Scatter(
    x = review_data.index,
    y = review_data.community_review,
    mode = 'lines',
    name = 'Community',
    line = dict(
    #    color = ('rgb(45, 142, 207, 0.4)'),
        width = 2),
    opacity=0.7
)
trace2 = go.Scatter(
    x = review_data.index,
    y = review_data.meta_review,
    mode = 'lines',
    name = 'Metacritic',
    line = dict(
    #    color = ('rgb(10, 173, 32, 0.4)'),
        width = 2),
    opacity=0.7
)
trace1 = go.Scatter(
    x = review_data.index,
    y = review_data.review,
    mode = 'lines',
    name = 'Playtime',
    line = dict(
    #    color = ('rgb(194, 176, 39, 0.4)'),
        width = 2),
    opacity=0.7
)
data = [trace0, trace1, trace2]

# Plot and embed in ipython notebook!
py.iplot(data)

In [42]:
community_reviews = reviews_add[["appid", "community_review", "meta_review"]]

In [43]:
community_reviews.head()

,appid,community_review,meta_review
0,7830,87,80
1,97330,89,77
2,108710,91,83
3,209000,85,74
4,72850,92,94


In [44]:
sorted_community = community_reviews.sort_values(by="appid")#.reset_index(inplace=True, drop=True)
sorted_community.reset_index(inplace=True, drop=True)

In [45]:
my_review_community_review = pd.merge(reviews_add, my_reviews, on="appid")

In [61]:
by_genre = my_review_community_review.groupby("genre").mean()
len(by_genre)

15

In [48]:
by_genre.index

Index([u'Accounting', u'Action', u'Adventure', u'Animation', u'Casual',
       u'Design', u'Free', u'Indie', u'Massively', u'RPG', u'Racing',
       u'Simulation', u'Sports', u'Strategy', u'Utilities'],
      dtype='object', name=u'genre')

In [66]:
by_genre[by_genre.index.isin(genres)].review

genre
Action        78.352891
Adventure     77.031587
Casual        73.985498
Free          77.366046
Indie         79.443063
Massively     91.617987
RPG           82.802510
Racing        74.101745
Simulation    81.443826
Sports        86.357967
Strategy      81.648482
Name: review, dtype: float64

In [62]:
genres = set(by_genre.index.tolist()) - set(["Accounting", "Design", "Utilities", "Animation"])

In [67]:
trace1 = go.Bar(
    x=by_genre[by_genre.index.isin(genres)].index,
    y=by_genre[by_genre.index.isin(genres)].community_review,
    name="Community"
)
trace2 = go.Bar(
    x=by_genre[by_genre.index.isin(genres)].index,
    y=by_genre[by_genre.index.isin(genres)].review,
    name="Playtime"
)
trace3 = go.Bar(
    x=by_genre[by_genre.index.isin(genres)].index,
    y=by_genre[by_genre.index.isin(genres)].meta_review,
    name="Metacritic"
)
data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group'
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)#, filename = 'Ratings') # will plot inline
#plot_url = py.plot(fig, filename='grouped-bar')

# Genres by Country?

Top countries: US, CA, GB, UA, DE

In [ ]:
with open("user_df.p", "rb") as f:
    user_data_df = cPickle.load(f)

In [79]:
countries = user_data_df[user_data_df["loccountrycode"].isin(["US","CA","GB","AU","DE"])]
country_ids = countries.steamid.unique()
country_user_items = user_items[user_items["steamid"].isin(country_ids)]
country_user_items = country_user_items.merge(countries[["steamid", "loccountrycode"]], on="steamid")
country_user_items = country_user_items.merge(item_data[["appid","genre"]], on="appid")

In [84]:
country_user_items.genre.value_counts()

Action        1983659
Strategy       192073
Indie          144213
RPG            142660
Adventure      124047
Casual          67327
Free            42607
Simulation      34379
Racing          24277
Massively        5833
Animation        5067
Sports           4275
Design           2410
Utilities        1921
Accounting       1251
Name: genre, dtype: int64

top genres: Action, Strategy, Indie, RPG, Adventure

In [85]:
country_user_items = country_user_items[country_user_items["genre"].isin(["Action", "Strategy","Indie","RPG","Adventure"])]

In [86]:
country_user_items.groupby(["loccountrycode", "genre"]).sum()["playtime_2weeks"]

loccountrycode  genre    
AU              Action        2476474
                Adventure       49305
                Indie          248945
                RPG            101076
                Strategy        92322
CA              Action        6824567
                Adventure      145445
                Indie          453509
                RPG            292676
                Strategy       268900
DE              Action        1622854
                Adventure       14343
                Indie          101094
                RPG             50252
                Strategy        34148
GB              Action        3490895
                Adventure       90473
                Indie          260732
                RPG            212926
                Strategy       202000
US              Action       33771906
                Adventure      733657
                Indie         2830545
                RPG           1714682
                Strategy      1416267
Name: playtime_2weeks, d

In [104]:
country_user_items.groupby(["loccountrycode"]).sum()

,appid,playtime_2weeks,playtime_forever,steamid,rating
loccountrycode,,,,,
AU,2.040566e+10,4218704,417915674,1.274798e+22,407642
CA,5.620635e+10,11617192,1238122597,3.586839e+22,1159313
DE,1.142330e+10,2644792,253277564,7.019513e+21,230962
GB,3.571059e+10,6095185,615617301,2.302425e+22,732901
US,3.266756e+11,59022497,6814759325,2.115281e+23,6779071


Look at these top five countries and the top five genre playtime, creating:

In [87]:
top_labels = ["Action", "Adventure", "Indie", "RPG",
              "Strategy"]

colors = ['rgba(178, 25, 45, .9)',
          'rgba(25, 122, 178, .9)', 
          'rgba(56, 200, 37, .9)',
          'rgba(113, 37, 200, .9)',
          'rgba(93, 32, 20, .9)']

x_data = [[83, 2, 7, 4, 4],
          [86, 2, 6, 3, 3],
          [82, 2, 6, 5, 5],
          [83, 2, 8, 3, 4],
          [89, 1, 6, 2, 2]]

y_data = ['United states',
          'Canada',
          'Great Britain',
          'Australia',
          'Germany']


traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=xd[i],
            y=yd,
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1,
                )
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    height=600,
    width=600,
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y', x=0.14, y=yd,
                                  xanchor='right', text=str(yd),
                                  font=dict(family='Arial', size=14,
                                            color='rgb(67, 67, 67)'),
                                  showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y', x=(xd[0] / 2), y=yd,
                                  text=str(xd[0]) + '%',
                                  font=dict(family='Arial', size=14,
                                            color='rgb(248, 248, 255)'),
                                  showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper', x=(xd[0] / 2),
                                      y=1.1, text=top_labels[0],
                                      font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                                      showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y', x=space
                                          + (xd[i] / 2), y=yd,
                                          text=str(xd[i]) + '%',
                                          font=dict(family='Arial', size=14,
                                                    color='rgb(248, 248, 255)'),
                                          showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper', x=space +
                                              (xd[i] / 2), y=1.1,
                                              text=top_labels[i],
                                              font=dict(family='Arial', size=14,
                                                        color='rgb(67, 67, 67)'),
                                              showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)

#plot_url = py.plot(fig, filename='bar-colorscale')
#fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'PlaytimeCountries') # will plot inline
#plot_url = py.plot(fig, filename='grouped-bar')

# `user_data`

In [93]:
with open("/home/ubuntu/banned_df.p", "rb") as f:
    banned_users_df = cPickle.load(f)
del banned_users_df["ban_status"]
del banned_users_df["user_info"]

In [94]:
with open("/home/ubuntu/user_df.p", "rb") as f:
    users_info_df = cPickle.load(f)

In [95]:
users_info_df = users_info_df[["lastlogoff", "loccountrycode", "locstatecode",\
                               "personaname", "personastate", "profilestate",\
                              "timecreated"]]
users_info_df.reset_index(inplace=True)
users_info_df.rename(columns={"index":"steamid"}, inplace=True)
user_data_df = pd.merge(banned_users_df, users_info_df, on="steamid")

In [ ]:
user_data_df.head(1)

In [ ]:
import time

In [ ]:
user_data_df["timecreated"] = ((time.time() - user_data_df["timecreated"]) / (3600 * 24)).round().astype(int)

In [ ]:
num_friends = []
for friend in user_data_df["friends"]:
    num_friends.append(len(friend))
user_data_df["num_friends"] = num_friends

In [ ]:
game_sample = user_data_df.game_info[:10]

In [ ]:
game_dict = {}
game_dict.values()

In [ ]:
num_games = []
num_played_total = []
for games in user_data_df["game_info"]:
    num_played = 0
    num_games.append(len(games))
    for game in games:
        if game["playtime_forever"] > 0:
            num_played += 1
    num_played_total.append(num_played)
user_data_df["num_games"] = num_games
user_data_df["num_played"] = num_played_total

In [ ]:
user_data_final = user_data_df[["steamid", "CommunityBanned", "VACBanned",
                               "personastate", "timecreated", "num_friends", "num_games",
                               "num_played"]]

# Combine Into Model
### as SFrames

In [ ]:
user_data_final.rename(columns={"steamid":"user_id"}, inplace=True)

In [ ]:
user_data_sf = gl.SFrame(user_data_final)
item_data_sf = gl.SFrame(item_data[["item_id","price_int","game_age", "review_nums","genre"]])

In [ ]:
user_item_sf = gl.SFrame({'user_id': user_items["steamid"],
                          'item_id': user_items["appid"],
                          'rating': user_items["rating"]})

In [ ]:
user_data_sf.save("/home/ubuntu/final_data_sets/user_data.csv", format="csv")
item_data_sf.save("/home/ubuntu/final_data_sets/item_data.csv", format="csv")
user_item_sf.save("/home/ubuntu/final_data_sets/user_item.csv", format="csv")

In [ ]:
model = gl.factorization_recommender.create(user_data_sf,
                                            target='rating',
                                            item_data=item_data_sf,
                                            user_data=user_data_sf)

In [ ]:
model = gl.factorization_recommender.create(user_item_sf,
                                            user_id='user_id',
                                            item_id='item_id',
                                            target='rating',
                                            user_data=user_data_sf,
                                            item_data=item_data_sf)

In [ ]:
folds = gl.cross_validation.KFold(sf, 5)
params = dict([('target', 'Y'),
               ('step_size', [0.01, 0.1]),
               ('max_depth', [5, 7])])
job = gl.grid_search.create(folds,
                            gl.boosted_trees_classifier.create,
                            params)
job.get_results()

In [ ]:
user_data_df[user_data_df["steamid"]==76561198151270288]

In [ ]:
user_data_df[user_data_df.NumberOfVACBans==6]

In [ ]:
similar_users = model.get_similar_users([76561198062169232], k=10)

In [ ]:
similar_users

In [ ]:
model.get_similar_items([72850])

In [ ]:
train, test = gl.recommender.util.random_split_by_user(user_item_sf)
m = gl.recommender.create(train, target='rating')
eval = m.evaluate(test)

In [ ]:
eval

In [ ]:
user_items[user_items["playtime_2weeks"]!=0].groupby("appid").count().sort_values(by="steamid", ascending=False)

In [ ]:
model.recommend(users=[76561198095427520])

In [ ]:
for dct in user_data_df[user_data_df.steamid==76561198095427520]["game_info"]:
    for game in dct:
        print str(game["name"]) + " : " + str(game["playtime_forever"])

In [ ]:
user_data_df[user_data_df.steamid==76561198096081584]

In [ ]:
user_data_df[user_data_df.steamid==76561198062748448]

In [ ]:
plt.figure(figsize=(9,7))
plt.hist(user_data_df.num_played / user_data_df.num_games, bins=20, normed=True)
plt.show()

In [ ]:
friends_sets = []
for user in user_data_df.friends:
    friend_set = set()
    for friend in user:
        friend_set.add(int(friend["steamid"]))
    friends_sets.append(friend_set)

In [ ]:
user_data_df["friend_sets"] = friends_sets

In [ ]:
similar_users = model.get_similar_users([76561198096081584], k=1172)

In [ ]:
similar_set = set(similar_users["similar"])

In [ ]:
len(similar_set)

In [ ]:
friends = user_data_df[user_data_df["steamid"]==76561198096081584]["friend_sets"].tolist()[0]

In [ ]:
num_in = 0 
for user in similar_set:
    if user in friends:
        num_in += 1

In [ ]:
num_in

In [133]:
with open("final_data_sets/full_games.json") as f:
    game_reviews = f.readlines()
game_reviews = map(lambda x: x.strip(), game_reviews)
game_reviews = "[" + ','.join(game_reviews) + "]"
full_games_df = pd.read_json(game_reviews)

In [135]:
del full_games_df["_id"]
full_games_df.head()

,appid,genre,price,release_year,reviews
0,221540,"[Tower Defense, Strategy, Indie, Sci-fi, Co-op...",14.99,"Sep 23, 2014",Very Positive
1,287140,"[Indie, Simulation, Massively Multiplayer, Cas...",4.99,"May 8, 2015",Mixed
2,370920,"[Action, Indie, Music, Fast-Paced, Difficult]",4.99,"Jul 9, 2015",Very Positive
3,330100,"[Early Access, Action, RPG, Indie, Simulation,...",14.99,"Nov 4, 2014",Very Positive
4,425870,"[Early Access, Sports, Action, Indie]",12.99,"Mar 4, 2016",Positive


In [161]:
data=[go.Histogram(x=map(lambda x: len(x), full_games_df[full_games_df["appid"].isin(action_games)]["genre"]))]
iplot(data)

In [ ]:
user_data_df[user_data_df["steamid"]==76561198020536208]

In [ ]:
76561197987075344 in user_data_df[user_data_df["steamid"]==76561198020536208].friend_sets.tolist()[0]

# Class to fit a newly looked up user

In [138]:
games = set(item_data.appid)

In [144]:
action_games = item_data[(item_data.appid.isin(games)) & (item_data.genre=="Action")]["appid"]

,appid,genre,price,release_year,reviews,game_age,review_nums,price_int,item_id,community_review,meta_review
0,10,Action,9.99,2000,Overwhelmingly Positive,16,8,10,10,97,88
1,20,Action,4.99,1999,Very Positive,17,7,5,20,84,84
2,30,Action,4.99,2003,Very Positive,13,7,5,30,90,79
3,50,Action,4.99,1999,Overwhelmingly Positive,17,8,5,50,95,95
4,70,Action,9.99,1998,Overwhelmingly Positive,18,8,10,70,96,96


In [165]:
meta_df[meta_df.meta_review==26]

,appid,meta_review
310,221020,26
